# Datasets Site info - from EDI to jrn_metabase

Code for populating the DataSetSites table in jrn_metabase is below. The EDI Solr search
API is a little wonky in that when site data, including geographic descriptions and 
coordinates queries, are requested the response doesn't contain everything in the EML.
Requesting geographic description will only return the geographicDescription element from the _first_ geographicCoverage found even if there are multiple geographicCoverages. If requesting coordinates, all boundingCoordinates elements found within the coverage element are returned. So, it makes sense to just count the spatial elements returned per datasetID.

In [1]:
import sys
sys.path.append('/home/greg/GitHub/')
import pyEDIutils.search as edi
import pandas as pd
import numpy as np

## Create a DataSets table with site/spatial info


In [2]:
# Query edi for all data packages in jrn scope and include spatial data
import importlib
importlib.reload(edi)
importlib.reload(edi.rq)
df, root = edi.request_search(fields=['packageid','title', 'site',
                                      'geographicdescription', 'coordinates'],
                              rows=1000, returnroot=True)

https://pasta.lternet.edu/package/search/eml?defType=edismax&q=%2A&fq=scope%3Aknb-lter-jrn&fl=packageid%2Ctitle%2Csite%2Cgeographicdescription%2Ccoordinates&sort=packageid%2Casc&rows=1000
More than 1 spatial entity per packageid, so just counting


In [3]:
df.head()

,packageid,title,site,geographicdescription,coordinates_ent
0,knb-lter-jrn.100.3,High resolution shrub cover raster maps of the...,jrn,This spatial dataset covers the Jornada Experi...,1
1,knb-lter-jrn.210001001.62,Plant cover on 2 x 2 meter rainfall runoff plo...,jrn,Creosotebush plots (C) located in the west baj...,2
2,knb-lter-jrn.210001002.76,Rainfall runoff and sediment deposition from 2...,jrn,Creosotebush plots (C) located in the west baj...,2
3,knb-lter-jrn.210001003.79,Rainfall runoff water chemistry from 2 x 2 met...,jrn,Creosotebush plots (C) located in the west baj...,2
4,knb-lter-jrn.210002001.127,Graduated rain gauge (GRG) precipitation obser...,jrn,"NPP is measured at 15 sites, spanning 5 vegeta...",1


In [4]:
# Split the dataset id and revisions out
df[['scope', 'datasetid', 'revision']] = df['packageid'].str.split('.',expand=True)
# Empty column for SiteID
df['SiteID'] = ''
len(df)

340

In [5]:
# Remove the met packages John/geovany use
import numpy as np
nonmet1 = ~df['datasetid'].str.contains('210437')
nonmet2 = ~df['datasetid'].str.contains('210548')
nonmet = np.logical_and(nonmet1, nonmet2)
df = df[nonmet]
len(df)

117

In [6]:
# For "None" geographic descriptins, assign a Headquarters siteID
df.loc[df.geographicdescription.isna(), 'SiteID'] = 'HQ_unspecified'
df.loc[df.geographicdescription.isna(), 'geographicdescription'] = 'HQ_unspecified'

In [7]:
# Match patterns in the geographic description to assign SiteIDs
test1 = df.geographicdescription.str.contains('west bajada', case=False)
test2 = df.geographicdescription.str.contains('bajada_west', case=False)
test3 = df.geographicdescription.str.contains('bajada west', case=False)
test = test1 | test2 |test3
df.loc[test, 'SiteID'] = 'Bajada_West'
test1 = df.geographicdescription.str.contains('east bajada', case=False)
test2 = df.geographicdescription.str.contains('bajada_east', case=False)
test = test1 | test2
df.loc[test, 'SiteID'] = 'Bajada_East'
test1 = df.geographicdescription.str.contains('basin floor', case=False)
test2 = df.geographicdescription.str.contains('basin_floor', case=False)
test = test1 | test2
df.loc[test, 'SiteID'] = 'Basin_Floor'
test1 = df.geographicdescription.str.contains('sand sheet', case=False)
test2 = df.geographicdescription.str.contains('sand_sheet', case=False)
test = test1 | test2
df.loc[test, 'SiteID'] = 'Sand Sheet'
test1 = df.geographicdescription.str.contains('transition', case=False)
df.loc[test1, 'SiteID'] = 'Transition_Zone'
test1 = df.geographicdescription.str.contains('NPP', case=False)
df.loc[test1, 'SiteID'] = 'NPP_Sites'
# We know what to call this one
test = df.datasetid=='100'
df.loc[test, 'SiteID'] = 'shrub_cover_raster_pkg100'

## What hasn't been identified?

In [9]:
df[df.SiteID=='']

,packageid,title,site,geographicdescription,coordinates_ent,scope,datasetid,revision,SiteID
8,knb-lter-jrn.210010001.75,Termite foraging data from bait mass loss at e...,jrn,The consumer site are located at 11 sites acro...,1,knb-lter-jrn,210010001,75,
20,knb-lter-jrn.210015002.44,Transect Soil Mineralization Potential (Field),jrn,Our study site is located in the northern Chih...,1,knb-lter-jrn,210015002,44,
21,knb-lter-jrn.210015003.48,Transect Soil Mineralization Potential (Initial),jrn,Our study site is located in the northern Chih...,1,knb-lter-jrn,210015003,48,
34,knb-lter-jrn.210121001.47,Biodiversity plots vegetation transects,jrn,Our study site is located in the northern Chih...,1,knb-lter-jrn,210121001,47,
36,knb-lter-jrn.210121006.67,The Effects of Changing Vegetative Composition...,jrn,Our study site is located in the northern Chih...,1,knb-lter-jrn,210121006,67,
39,knb-lter-jrn.210126001.113,LTER Weather Station daily summary climate data,jrn,Our study site is located in the northern Chih...,1,knb-lter-jrn,210126001,113,
45,knb-lter-jrn.210262001.8,Spring and Fall plant cover across grassland-s...,jrn,Pastures 9 and 12A of the Jornada Experimental...,1,knb-lter-jrn,210262001,8,
46,knb-lter-jrn.210262004.93,Annual Aboveground Net Primary Productivity by...,jrn,Pastures 9 and 12A of the Jornada Experimental...,1,knb-lter-jrn,210262004,93,
47,knb-lter-jrn.210262005.9,Total Annual Aboveground Net Primary Productiv...,jrn,Pastures 9 and 12A of the Jornada Experimental...,1,knb-lter-jrn,210262005,9,
48,knb-lter-jrn.210262008.139,Rodent capture data across grassland-shrubland...,jrn,Pastures 9 and 12A of the Jornada Experimental...,1,knb-lter-jrn,210262008,139,


In [10]:
# Assign these to Headquarters too...
df.loc[df.SiteID=='', 'SiteID'] = 'HQ_unspecified'

## For datasets with >1 geographicCoverage

We counted coordinate entities, now we need to duplicate the rows for datasets > 1 and
concatenate them onto the dataframe again.

In [12]:
# These datsets have multiple geoCoverages
mults = df.coordinates_ent > 1
df_rep = df[mults].copy() # Copy the rows
df_rep.SiteID = 'Unknown_placeholder' # Assign a placeholder (in DB) to fill in later
df_rep.head()

,packageid,title,site,geographicdescription,coordinates_ent,scope,datasetid,revision,SiteID
1,knb-lter-jrn.210001001.62,Plant cover on 2 x 2 meter rainfall runoff plo...,jrn,Creosotebush plots (C) located in the west baj...,2,knb-lter-jrn,210001001,62,Unknown_placeholder
2,knb-lter-jrn.210001002.76,Rainfall runoff and sediment deposition from 2...,jrn,Creosotebush plots (C) located in the west baj...,2,knb-lter-jrn,210001002,76,Unknown_placeholder
3,knb-lter-jrn.210001003.79,Rainfall runoff water chemistry from 2 x 2 met...,jrn,Creosotebush plots (C) located in the west baj...,2,knb-lter-jrn,210001003,79,Unknown_placeholder
14,knb-lter-jrn.210012001.139,Raw neutron counts from a soil water content h...,jrn,Bajada_West,2,knb-lter-jrn,210012001,139,Unknown_placeholder
15,knb-lter-jrn.210012002.127,Soil volumetric water content calculated from ...,jrn,Bajada_West,2,knb-lter-jrn,210012002,127,Unknown_placeholder


In [13]:
# Concatenate these rows to our dataframe
df2 = pd.concat([df, df_rep])
print(df.shape)
df2.shape

(117, 9)


(144, 9)

## Prepare table for metabase

In [15]:
# Format the dataframe to look like the DataSetSites table in jrn_metabase
df2['EntitySortOrder'] = 0
df2['GeoCoverageSortOrder'] = 1
df_in = df2.loc[:,['datasetid','EntitySortOrder','SiteID','GeoCoverageSortOrder']]
df_in.head()

,datasetid,EntitySortOrder,SiteID,GeoCoverageSortOrder
0,100,0,shrub_cover_raster_pkg100,1
1,210001001,0,Bajada_West,1
2,210001002,0,Bajada_West,1
3,210001003,0,Bajada_West,1
4,210002001,0,NPP_Sites,1


In [32]:
# Some of these are in the database already - remove from incoming table
print(len(df_in))
test = ((df_in.datasetid=='100') | (df_in.datasetid=='210001001') | 
         (df_in.datasetid=='210001002') | (df_in.datasetid=='210001003') |
         (df_in.datasetid=='210002001'))
df_in2 = df_in[~test]
len(df_in2)

144


136

## Now insert the table

In [33]:
# Establish database connection
sys.path.append('../')
import py2pg.connect as connect
conn = connect.connect('../jrn_metabase_dev.conn.json')

In [34]:
# Do a test query of the database
sql = 'select * from lter_metabase."DataSetSites";'
dat = pd.read_sql_query(sql, conn)

In [35]:
dat.head()

,DataSetID,EntitySortOrder,SiteID,GeoCoverageSortOrder
0,99021,0,ABUR_reef,1
1,99021,0,AQUE_reef,1
2,99021,0,MOHK_reef,1
3,100,1,shrub_cover_raster_pkg100,1
4,210001001,1,Bajada_West,1


In [36]:
import py2pg.populate as pop

In [37]:
#Use function to load ent_in
pop.copy_from_file(conn, df_in2, 'lter_metabase."DataSetSites"') # copy the dataframe to SQL
# Close the database connection
conn.close()

copy_from_file() done
